#Multi-User ChatBot from Scratch by langGraph

Tasks :- 1) build simple agent using langgraph

2) build the agent with tools (WebSearch and Weather api)

3) build the multi user chatbot agent with tools.


In [1]:
!pip install httpx
!pip install langchain
!pip install langgraph
!pip install langchain_community
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires reque

In [2]:
from getpass import getpass
OPENAI_API_KEY=getpass("enter the openai key:-")
TAVILY_API_KEY=getpass("enter the tavily api key:-")
WEATHER_API_KEY=getpass("enter the weather api key:-")

enter the openai key:-··········
enter the tavily api key:-··········
enter the weather api key:-··········


In [3]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY
os.environ["WEATHER_API_KEY"]=WEATHER_API_KEY
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY

In [4]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o",temperature=0)

#1) build simple agent using langgraph

In [15]:
from langgraph.graph import StateGraph
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class GraphState(TypedDict):
  messages:Annotated[list,add_messages]

graph=StateGraph(GraphState)


In [12]:
def chatgpt(state:GraphState):
  return {"messages":[llm.invoke(state["messages"])]}

In [16]:
graph.add_node("llm",chatgpt)
graph.set_finish_point("llm")
graph.set_entry_point("llm")

In [17]:
agent=graph.compile()

In [18]:
response=agent.invoke({"messages":("user","give me syllabus of langgraph.")})

In [21]:
print(response["messages"][-1].content)

It seems like you might be referring to a specific course or subject called "LangGraph," but there isn't a widely recognized course or subject by that name as of my last update. It's possible that "LangGraph" could be a specialized course offered by a particular institution or a new subject that has emerged after my last update.

To get the most accurate and detailed syllabus for "LangGraph," I recommend the following steps:

1. **Institution Website**: Check the website of the institution offering the course. They often have detailed syllabi available for students.

2. **Course Catalog**: Look for a course catalog or academic bulletin from the institution. These documents typically include course descriptions and syllabi.

3. **Contact the Instructor**: If you have contact information for the instructor or department offering the course, reaching out directly can provide you with the most specific and up-to-date information.

4. **Online Platforms**: If "LangGraph" is a course offered

__We can easily see that our llm not have data on this topic let's build the agent with web search capability.__

#2) build the agent with tools (WebSearch and Weather api)

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import requests

tv_search=TavilySearchResults(max_results=3,search_depth='advanced',max_tokens=10000)

/tmp/ipython-input-998168330.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tv_search=TavilySearchResults(max_results=3,search_depth='advanced',max_tokens=10000)


In [23]:
@tool
def webSearch(query:str):
  """this is a tool for search any information on web."""
  return tv_search.invoke(query)

@tool
def getWeather(query:str):
  """this is a tool for getting the current information of weather or environment."""
  BASE_URL="http://api.weatherapi.com/v1/current.json"
  comp_url=f"{BASE_URL}?key={WEATHER_API_KEY}&q={query}"
  response=requests.get(comp_url)
  data=response.json()
  if data.get("location"):
    return data
  else:
    return "Weather data not found !"


In [24]:
llm_with_tool=llm.bind_tools([getWeather,webSearch])

In [25]:
def new_chatgpt(state:GraphState):
  return {"messages":[llm_with_tool.invoke(state["messages"])]}

In [37]:
new_graph=StateGraph(GraphState)
new_graph.add_node("llm",new_chatgpt)

In [38]:
from langgraph.prebuilt import ToolNode,tools_condition
new_graph.add_node("tools",ToolNode(tools=[webSearch,getWeather]))
new_graph.set_entry_point("llm")
new_graph.add_conditional_edges("llm",tools_condition,{"tools":"tools","__end__":"__end__"})
new_graph.add_edge("tools","llm")

In [39]:
new_agent=new_graph.compile()
response=new_agent.invoke({"messages":[("system","you are the assistant. if u don't know any information then use the associated tools."),("user","give me syllabus of langgraph.")]})
print(response["messages"][-1].content)


The LangGraph syllabus includes a comprehensive set of topics designed to enhance your skills in AI and LLM (Large Language Model) applications. Here's a summary of the key components:

1. **Core LangGraph Concepts**: Understanding chains, Directed Acyclic Graphs (DAGs), and their applications.
2. **LangGraph vs Traditional Agents**: Building smarter, modular AI workflows.
3. **Retrieval-Augmented Generations (RAGs) and CRAGs**: Mastering these techniques using LangGraph.
4. **Multi-Agent Systems**: Creating complex, interactive AI agents.
5. **Real-World Applications**: Engaging in hands-on projects to apply theoretical knowledge.

For more detailed information, you can explore resources like the [YouTube video on the LangGraph course syllabus](https://www.youtube.com/watch?v=zcC8eK6grqw) or the [LangGraph tutorial on DataCamp](https://www.datacamp.com/tutorial/langgraph-tutorial).


In [40]:
print(response)

{'messages': [SystemMessage(content="you are the assistant. if u don't know any information then use the associated tools.", additional_kwargs={}, response_metadata={}, id='df1d38e8-9b97-4637-af54-7b8225f5da93'), HumanMessage(content='give me syllabus of langgraph.', additional_kwargs={}, response_metadata={}, id='c2c283ef-9842-437a-b90f-9394322a3c36'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4sHSUqWRyDKEV5M6IMoxtBUx', 'function': {'arguments': '{"query":"LangGraph syllabus"}', 'name': 'webSearch'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 100, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f33640a400', 'id': 'chatcmpl-CLPadSHgenNzVkeMOq8ha3KYAa8

In [41]:
response=new_agent.invoke({"messages":[("system","you are the assistant. if u don't know any information then use the associated tools."),("user","give me current weather of ajmer.")]})
print(response["messages"][-1].content)


The current weather in Ajmer, Rajasthan, India is as follows:

- **Temperature:** 30.5°C (86.9°F)
- **Condition:** Moderate or heavy rain shower
- **Wind:** 6.0 mph (9.7 kph) from the SSE
- **Humidity:** 56%
- **Cloud Cover:** 78%
- **Precipitation:** 3.16 mm
- **Feels Like:** 33.3°C (91.9°F)
- **Visibility:** 7.0 km
- **UV Index:** 7.6

![Weather Icon](https://cdn.weatherapi.com/weather/64x64/day/356.png)


In [45]:
print(response["messages"][2]) #tool call

content='' additional_kwargs={'tool_calls': [{'id': 'call_Vj6ODSIdO24Nohfhvx13Hvxi', 'function': {'arguments': '{"query":"Ajmer"}', 'name': 'getWeather'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 101, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f33640a400', 'id': 'chatcmpl-CLPc6ieyIXnCrrJFegmrxswmmn0nG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--0f17bc3f-b6f8-47ec-b7fc-632ad7f54124-0' tool_calls=[{'name': 'getWeather', 'args': {'query': 'Ajmer'}, 'id': 'call_Vj6ODSIdO24Nohfhvx13Hvxi', 'type': 'tool_call'}] usage_metadata={'input_tokens': 101, 'output_tokens': 15, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read

In [46]:
response=new_agent.invoke({"messages":[("system","you are the assistant. if u don't know any information then use the associated tools."),("user","give me more information about this city.")]})
print(response["messages"][-1].content)

Could you please specify the name of the city you would like more information about?


__WE can see , out agent doesn't know about previous history of conversation.__
# 3) Multi-user agent chatbot

In [48]:
from langgraph.checkpoint.memory import InMemorySaver
db=InMemorySaver()

In [49]:
final_agent=new_graph.compile(checkpointer=db)

In [53]:
response=final_agent.invoke({"messages":[("system","you are the assistant. if u don't know any information then use the associated tools."),("user","give me current weather of ajmer.")]}, {'configurable': { 'thread_id':"abc123"}})
print(response["messages"][-1].content)


The current weather in Ajmer, Rajasthan, India is as follows:

- **Temperature:** 30.5°C (86.9°F)
- **Condition:** Moderate or heavy rain shower
- **Wind:** 6.0 mph (9.7 kph) from the SSE
- **Humidity:** 56%
- **Pressure:** 1003.0 mb
- **Precipitation:** 3.16 mm
- **Visibility:** 7.0 km
- **UV Index:** 7.6
- **Feels Like:** 33.3°C (91.9°F)

![Weather Icon](//cdn.weatherapi.com/weather/64x64/day/356.png)


In [54]:
response=final_agent.invoke({"messages":[("system","you are the assistant. if u don't know any information then use the associated tools."),("user","give me more information about this city.")]}, {'configurable': { 'thread_id':"abc123"}})
print(response["messages"][-1].content)


Ajmer is a city located in the north-western Indian state of Rajasthan. It serves as the administrative headquarters of the Ajmer district and division. Known as the "Heart of Rajasthan," Ajmer is situated at the center of the state. Here are some key details about Ajmer:

- **Location:** Ajmer is located 130 km southwest of Jaipur and 14 km from the sacred town of Pushkar. It lies among the Aravalli ranges.
- **Historical Significance:** The city is renowned for the Ajmer Sharif Dargah, a revered Muslim shrine dedicated to the Sufi saint Moinuddin Chishti, attracting millions of pilgrims from all faiths. Another notable site is the Adhai Din Ka Jhonpra, originally a Sanskrit college transformed into a mosque in the 12th century.
- **Cultural Heritage:** Ajmer is a vibrant blend of religions, communities, and cultures, epitomizing the diversity of Indian heritage. The city is known for its architectural wonders and spiritual legacy.
- **Economy:** Ajmer is a trade center for salt, mica

__We can see that now our agent can remmeber the previous conversation according to session id.__